# Auto log a single ID


In [167]:
import plotly.express as px
import pandas as pd

import xlsxwriter
import glob
import os

# Import plotly.express to utilize map

### Import your groupMetadataID below


In [168]:
gmID = '3a2a78cc-db21-11ee-a158-97f8443fd730'

# set to False if data is stored as .csv file.
# otherwise, set to True

if os.path.isfile(f"./data/chassis/{gmID}.csv"):
    queryChassis = False
    print("chassis csv file found!")
else:
    queryChassis = True
    print("no chassis csv. Query for data necessary.")

if os.path.isfile(f"./data/best_pose/{gmID}.csv"):
    queryPose = False
    print("best_pose csv file found!")
else:
    queryPose = True
    print("no best_ose csv. Query for data necessary.")

if os.path.isfile(f"./data/metadata/{gmID}.csv"):
    queryMeta = False
    print("metadata csv file found!")
else:
    queryMeta = True
    print("no metadata csv. Query for data necessary.")

chassis csv file found!
best_pose csv file found!
metadata csv file found!


In [169]:
from boto3.dynamodb.conditions import Key, Attr
import boto3

if queryPose or queryChassis or queryMeta:
    # Get the service resource
    dynamodb = boto3.resource('dynamodb')
    # Instantiate a table resource object
    table = dynamodb.Table('ads_passenger_processed')
    table_meta = dynamodb.Table('ads_passenger_processed_metadata')

In [170]:

if not os.path.exists(f"./data/best_pose"):
        os.makedirs(f"./data/best_pose")

if queryPose:

    # query arguments for best_pose data
    keywords = dict(
        IndexName='topic-index',
        KeyConditionExpression=Key('topic').eq(
            '/apollo/sensor/gnss/best_pose'),
        ProjectionExpression="groupMetadataID, #t, solStatus, solType, latitudeStdDev, longitudeStdDev, numSatsTracked, numSatsMulti, numSatsInSolution, differentialAge, solutionAge, latitude, longitude",
        Limit=1500,
        ExpressionAttributeNames={'#t': 'time'},
        FilterExpression=Key('groupMetadataID').eq(f'{gmID}')

    )

    # run initial query and coerce results into a dataframe
    res = table.query(**keywords)
    df_pose = pd.DataFrame.from_dict(
        pd.json_normalize(res['Items']), orient='columns')

    done = False
    leek = res["LastEvaluatedKey"]

    while not done:
        try:
            leek = res["LastEvaluatedKey"]
            keywords["ExclusiveStartKey"] = leek
            res = table.query(**keywords)
            tmpDF = pd.DataFrame.from_dict(
                pd.json_normalize(res['Items']), orient='columns')
            df_pose = pd.concat([df_pose, tmpDF])
        except KeyError:
            print("Done querying best_pose data")
            done = True

    df_pose.to_csv(f"./data/best_pose/{gmID}.csv")
    print(f"Saved best_pose data at ./data/best_pose/{gmID}.csv")


else:
    df_pose = pd.read_csv(f"./data/best_pose/{gmID}.csv")
    print("Loaded best_pose data")

df_pose.shape

Loaded best_pose data


(2001, 13)

In [171]:
if not os.path.exists(f"./data/chassis"):
        os.makedirs(f"./data/chassis")
        
if queryChassis:
    keywords = dict(
        IndexName='topic-index',
        KeyConditionExpression=Key('topic').eq(
            '/apollo/canbus/chassis'),
        Limit=2000,
        FilterExpression=Key('groupMetadataID').eq(gmID)
    )

    res = table.query(**keywords)
    df_chassis = pd.DataFrame.from_dict(
        pd.json_normalize(res['Items']), orient='columns')

    done = False
    leek = res["LastEvaluatedKey"]

    while not done:
        try:
            leek = res["LastEvaluatedKey"]
            keywords["ExclusiveStartKey"] = leek
            res = table.query(**keywords)
            tmpDF = pd.DataFrame.from_dict(
                pd.json_normalize(res['Items']), orient='columns')
            df_chassis = pd.concat([df_chassis, tmpDF])

        except KeyError:
            print(f"done querying for chassis data")
            done = True

    df_chassis.to_csv(f"./data/chassis/{gmID}.csv")
    print(f"Saved chassis data at ./data/chassis/{gmID}.csv")
else:
    df_chassis = pd.read_csv(f"./data/chassis/{gmID}.csv")
    print("Loaded chassis data")

print(df_chassis.shape)

Loaded chassis data
(31903, 31)


In [172]:
if not os.path.exists(f"./data/metadata"):
        os.makedirs(f"./data/metadata")

if queryMeta:

    res = table_meta.query(
        IndexName='groupMetadataID-index',
        KeyConditionExpression=Key("groupMetadataID").eq(gmID),
        ProjectionExpression="groupMetadataID, #o.Weather, #o.#m, #o.Notes",
        Limit=1500,
        ExpressionAttributeNames={"#o": "other", "#m": "Map"},
    )

    df_meta = pd.DataFrame.from_dict(
        pd.json_normalize(res['Items']), orient='columns').drop_duplicates()

    df_meta.to_csv(f"./data/metadata/{gmID}.csv")
    print(f"Saved metadata data at ./data/metadata/{gmID}.csv")

else:
    df_meta = pd.read_csv(f"./data/metadata/{gmID}.csv")
    print("Loaded metadata")


print(df_meta.shape)

Loaded metadata
(1, 8)


The output of this box will be messed up. It automatically sets row 0 to True. Not sure why, but it doesn't count as a transition in the end.
We are also counting emergency mode as MANUAL


In [173]:
# read the timeSorted csv
pts = df_chassis.copy()
pts = pts.sort_values('time')
# reduce df to only time and drivingMode
pts = pts[["time", "drivingMode"]]
# function to tell us what state transition it is


def getState(old, new):
    if old == "COMPLETE_AUTO_DRIVE" and new == "COMPLETE_MANUAL":
        return "A->M"
    elif old == "COMPLETE_AUTO_DRIVE" and new == "EMERGENCY_MODE":
        return "A->M"
    elif old == "COMPLETE_MANUAL" and new == "COMPLETE_AUTO_DRIVE":
        return "M->A"
    elif old == "EMERGENCY_MODE" and new == "COMPLETE_AUTO_DRIVE":
        return "M->A"
    else:
        return "NONE"


# we shift the drivingMode column down by 1
pts["shift"] = pts["drivingMode"].shift()
# create transition field
pts['transition'] = (pts["drivingMode"] != pts["shift"])
# set Transition type to NONE for all
pts['transitionType'] = "NONE"
# Set first transition to False, since it was auto True
pts.loc[0, 'transition'] = False
# print(pts['transition'])
# Show counts for transitions
print(pts['transition'].value_counts())
# iterate over all rows and find the ones where transition is true. Then we compare shift and dM to see what
# transition Type we have.
for index, row in pts.iterrows():
    if pts['transition'][index] == True:
        old = pts['shift'][index]
        new = pts['drivingMode'][index]
        pts.loc[index, "transitionType"] = getState(old, new)
print(pts['transitionType'].value_counts())
pts = pts[['time', 'drivingMode', 'transition', 'transitionType']]
pts = pts.sort_values('time')
pts['time'] = pts['time'].astype(float)
pts['time'] = pts['time'].astype('datetime64[ns]')
pts['time'] = pts['time'].astype('datetime64[s]')
pts[['time']]

# show(pts)

transition
False    31854
True        49
Name: count, dtype: int64


transitionType
NONE    31871
M->A       16
A->M       16
Name: count, dtype: int64


,time
3092,2023-08-17 15:14:25
31671,2023-08-17 15:14:25
14369,2023-08-17 15:14:25
2187,2023-08-17 15:14:25
19724,2023-08-17 15:14:25
...,...
2064,2023-08-17 15:47:47
30421,2023-08-17 15:47:47
1938,2023-08-17 15:47:47
12877,2023-08-17 15:47:47


In [174]:
bestPos = df_pose.copy()
bestPos = bestPos.sort_values('time')
bestPos['time'] = bestPos['time'].astype(float)
bestPos['time'] = bestPos['time'].astype('datetime64[ns]')
bestPos['time'] = bestPos['time'].astype('datetime64[s]')
bestPos = bestPos[['time', 'latitude', 'longitude']]
merged_df = pd.merge(pts, bestPos, on='time')
merged_df = merged_df.drop_duplicates(
    subset=['time', 'transitionType']).reset_index()
# remove all NONE transition columns
automatedDF = merged_df[(merged_df['transitionType'] != 'NONE')]
# show(automatedDF)
# create a pairing system so we know start and end positions
pair = True
# list to hold the entries
documents = []
# iterate over the list
for index, row in automatedDF.iterrows():
    if pair == False:
        print(f"{index}:", row['transitionType'])
        # log the end time of transitioning back into automatic
        endTime = row['time']
        endingLoc = [row['latitude'], row['longitude']]
        # create our document here since we know we have a pair at the moment
        document = {
            'gmID': gmID,
            'Start Time': startTime,
            'Stop Time': endTime,
            'Start Lat': startingLoc[0],
            'Start Long': startingLoc[1],
            'Start Index': startingIndex,
            'Stop Lat': endingLoc[0],
            'Stop Long': endingLoc[1],
            'Stop Index': index,
            'Notes': '',
            'Features Present': '',
            'Image': ''
        }
        documents.append(document)
        pair = True
    elif row['transitionType'] == 'A->M':
        print(f"{index}:", row['transitionType'])
        startTime = row['time']
        startingLoc = [row['latitude'], row['longitude']]
        startingIndex = index
        # print(f'{startTime} location: {startingLoc}')
        pair = False
    else:
        print(f"no pair for {index}:", row['transitionType'])
        # raise Exception("Error!")


if not os.path.exists(f"./data/logged"):
    os.makedirs(f"./data/logged")
# print(documents)
autoOutput = pd.DataFrame(documents)

# heatmap = pd.read_csv("heatmap_4.csv")
# heatmap = pd.concat([heatmap, merged_df])
# heatmap = heatmap.drop(['Unnamed: 0'], axis='columns')
# heatmap.to_csv("heatmap6.csv")
# heatmap

no pair for 52: M->A
137: A->M
169: M->A
226: A->M
236: M->A
289: A->M
352: M->A
372: A->M
378: M->A
398: A->M
413: M->A
500: A->M
506: M->A
568: A->M
575: M->A
777: A->M
781: M->A
806: A->M
814: M->A
875: A->M
889: M->A
892: A->M
896: M->A
912: A->M
915: M->A
949: A->M
954: M->A
978: A->M
991: M->A
992: A->M
998: M->A
1241: A->M


In [175]:
merged_df

,index,time,drivingMode,transition,transitionType,latitude,longitude
0,0,2023-08-17 15:14:27,EMERGENCY_MODE,False,NONE,39.329602,-82.127926
1,30,2023-08-17 15:14:28,EMERGENCY_MODE,False,NONE,39.329603,-82.127926
2,60,2023-08-17 15:14:29,EMERGENCY_MODE,False,NONE,39.329602,-82.127926
3,91,2023-08-17 15:14:30,EMERGENCY_MODE,False,NONE,39.329603,-82.127926
4,121,2023-08-17 15:14:31,EMERGENCY_MODE,False,NONE,39.329602,-82.127926
...,...,...,...,...,...,...,...
1264,43707,2023-08-17 15:47:43,EMERGENCY_MODE,False,NONE,39.330517,-82.128211
1265,43737,2023-08-17 15:47:44,EMERGENCY_MODE,False,NONE,39.330517,-82.128211
1266,43767,2023-08-17 15:47:45,EMERGENCY_MODE,False,NONE,39.330517,-82.128211
1267,43786,2023-08-17 15:47:46,EMERGENCY_MODE,False,NONE,39.330517,-82.128211


In [176]:
autoOutput

,gmID,Start Time,Stop Time,Start Lat,Start Long,Start Index,Stop Lat,Stop Long,Stop Index,Notes,Features Present,Image
0,3a2a78cc-db21-11ee-a158-97f8443fd730,2023-08-17 15:16:50,2023-08-17 15:17:25,39.320644,-82.107480,137,39.319728,-82.105326,169,,,
1,3a2a78cc-db21-11ee-a158-97f8443fd730,2023-08-17 15:18:57,2023-08-17 15:19:05,39.326691,-82.102714,226,39.326623,-82.101713,236,,,
2,3a2a78cc-db21-11ee-a158-97f8443fd730,2023-08-17 15:20:52,2023-08-17 15:22:44,39.327251,-82.101367,289,39.329284,-82.102271,352,,,
3,3a2a78cc-db21-11ee-a158-97f8443fd730,2023-08-17 15:23:09,2023-08-17 15:23:20,39.327785,-82.102740,372,39.327755,-82.102759,378,,,
4,3a2a78cc-db21-11ee-a158-97f8443fd730,2023-08-17 15:23:56,2023-08-17 15:24:22,39.328951,-82.104938,398,39.329446,-82.107703,413,,,
5,3a2a78cc-db21-11ee-a158-97f8443fd730,2023-08-17 15:27:26,2023-08-17 15:27:30,39.330921,-82.125786,500,39.330467,-82.127142,506,,,
6,3a2a78cc-db21-11ee-a158-97f8443fd730,2023-08-17 15:28:58,2023-08-17 15:29:14,39.330473,-82.128025,568,39.331886,-82.129938,575,,,
7,3a2a78cc-db21-11ee-a158-97f8443fd730,2023-08-17 15:34:32,2023-08-17 15:34:35,39.375829,-82.134407,777,39.376198,-82.134524,781,,,
8,3a2a78cc-db21-11ee-a158-97f8443fd730,2023-08-17 15:35:08,2023-08-17 15:35:15,39.378515,-82.135271,806,39.379052,-82.135454,814,,,
9,3a2a78cc-db21-11ee-a158-97f8443fd730,2023-08-17 15:36:29,2023-08-17 15:36:56,39.385880,-82.138236,875,39.387741,-82.140652,889,,,


In [177]:
i = 0
print(i)
print()
start = autoOutput.iloc[i]["Start Index"] - \
    min(autoOutput.iloc[i]["Start Index"], 5)
stop = autoOutput.iloc[i]["Stop Index"] + \
    min(merged_df.shape[0] - autoOutput.iloc[i]["Stop Index"], 5)

print(autoOutput.iloc[i]["Start Index"], "vs", 5)
print((merged_df.shape[0] - 1) - autoOutput.iloc[i]["Stop Index"], "vs", 5)

print()
print(merged_df.shape[0] -1)
print("Start:", start)
print("Stop:", stop)

0

137 vs 5
1099 vs 5

1268
Start: 132
Stop: 174


### Mapping


In [178]:
# first full transition
tran = merged_df[start:stop]
tran.head()

,index,time,drivingMode,transition,transitionType,latitude,longitude
132,4068,2023-08-17 15:16:46,COMPLETE_AUTO_DRIVE,False,NONE,39.321171,-82.108258
133,4098,2023-08-17 15:16:47,COMPLETE_AUTO_DRIVE,False,NONE,39.321040,-82.108064
134,4128,2023-08-17 15:16:48,COMPLETE_AUTO_DRIVE,False,NONE,39.320908,-82.107869
135,4159,2023-08-17 15:16:49,COMPLETE_AUTO_DRIVE,False,NONE,39.320777,-82.107674
136,4189,2023-08-17 15:16:50,COMPLETE_AUTO_DRIVE,False,NONE,39.320644,-82.107480


In [179]:
# Show the first transition
fig = px.scatter_mapbox(tran, lat="latitude", lon="longitude",
                        hover_data=["time", "drivingMode", "transitionType"], color="drivingMode", zoom=17, height=700, width=1000,
                        color_discrete_map={
                            "COMPLETE_MANUAL": "#601A4A",
                            "COMPLETE_AUTO_DRIVE": "#63ACBE",
                            "EMERGENCY_MODE": "#EE442F",
                        }
                        )

fig.update_traces(marker=dict(size=8))

# Layout settings
fig.update_layout(
    mapbox_style="open-street-map",
    margin={"r": 0, "t": 0, "l": 0, "b": 0},
)


# Display the map
# fig.show()

/home/vhepola_linux/do-env/lib/python3.10/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [180]:
if not os.path.exists(f"./images/{gmID}"):
    os.makedirs(f"./images/{gmID}")

links = []
for index, row in autoOutput.iterrows():
    start = autoOutput.iloc[index]["Start Index"] - \
        min(autoOutput.iloc[index]["Start Index"], 5)

    stop = autoOutput.iloc[index]["Stop Index"] + \
        min(merged_df.shape[0] - 1 - autoOutput.iloc[index]["Stop Index"], 5)

    print("Start:", start, "Stop:", stop)
    temp_tran = merged_df[start:stop]

    fig = px.scatter_mapbox(temp_tran, lat="latitude", lon="longitude",
                            hover_data=["time", "drivingMode", "transitionType"], color="drivingMode", zoom=17, height=700, width=1000,
                            color_discrete_map={
                                "COMPLETE_MANUAL": "#601A4A",
                                "COMPLETE_AUTO_DRIVE": "#63ACBE",
                                "EMERGENCY_MODE": "#EE442F",
                            }
                            )

    # Layout settings
    fig.update_layout(
        mapbox_style="open-street-map",
        margin={"r": 0, "t": 0, "l": 0, "b": 0},
    )

    fig.update_traces(marker=dict(size=8))

    name = f"./images/{gmID}/tran_{index}.jpeg"
    links.append(name)

    fig.write_image(name, engine="kaleido")

Start: 132 Stop: 174


/home/vhepola_linux/do-env/lib/python3.10/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



Start: 221 Stop: 241


/home/vhepola_linux/do-env/lib/python3.10/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



Start: 284 Stop: 357


...

/home/vhepola_linux/do-env/lib/python3.10/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



Start: 973 Stop: 996


/home/vhepola_linux/do-env/lib/python3.10/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



Start: 987 Stop: 1003


/home/vhepola_linux/do-env/lib/python3.10/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [181]:
#autoOutput["Image"] = links
autoOutput
#links

,gmID,Start Time,Stop Time,Start Lat,Start Long,Start Index,Stop Lat,Stop Long,Stop Index,Notes,Features Present,Image
0,3a2a78cc-db21-11ee-a158-97f8443fd730,2023-08-17 15:16:50,2023-08-17 15:17:25,39.320644,-82.107480,137,39.319728,-82.105326,169,,,
1,3a2a78cc-db21-11ee-a158-97f8443fd730,2023-08-17 15:18:57,2023-08-17 15:19:05,39.326691,-82.102714,226,39.326623,-82.101713,236,,,
2,3a2a78cc-db21-11ee-a158-97f8443fd730,2023-08-17 15:20:52,2023-08-17 15:22:44,39.327251,-82.101367,289,39.329284,-82.102271,352,,,
3,3a2a78cc-db21-11ee-a158-97f8443fd730,2023-08-17 15:23:09,2023-08-17 15:23:20,39.327785,-82.102740,372,39.327755,-82.102759,378,,,
4,3a2a78cc-db21-11ee-a158-97f8443fd730,2023-08-17 15:23:56,2023-08-17 15:24:22,39.328951,-82.104938,398,39.329446,-82.107703,413,,,
5,3a2a78cc-db21-11ee-a158-97f8443fd730,2023-08-17 15:27:26,2023-08-17 15:27:30,39.330921,-82.125786,500,39.330467,-82.127142,506,,,
6,3a2a78cc-db21-11ee-a158-97f8443fd730,2023-08-17 15:28:58,2023-08-17 15:29:14,39.330473,-82.128025,568,39.331886,-82.129938,575,,,
7,3a2a78cc-db21-11ee-a158-97f8443fd730,2023-08-17 15:34:32,2023-08-17 15:34:35,39.375829,-82.134407,777,39.376198,-82.134524,781,,,
8,3a2a78cc-db21-11ee-a158-97f8443fd730,2023-08-17 15:35:08,2023-08-17 15:35:15,39.378515,-82.135271,806,39.379052,-82.135454,814,,,
9,3a2a78cc-db21-11ee-a158-97f8443fd730,2023-08-17 15:36:29,2023-08-17 15:36:56,39.385880,-82.138236,875,39.387741,-82.140652,889,,,


In [182]:
writer = pd.ExcelWriter(f"./data/logged/{gmID}.xlsx", engine="xlsxwriter")
autoOutput.to_excel(writer, sheet_name='Sheet1')

workbook = writer.book
worksheet = writer.sheets["Sheet1"]
# resize cells
# worksheet.set_column('B1:B5', 7)
# worksheet.set_column(first_col=1, last_col=10, width=20)
worksheet.autofit()

worksheet.set_column(first_col=2, last_col=3, width=25)
worksheet.set_column(first_col=10, last_col=11, width=50)
worksheet.set_column(first_col=12, last_col=12, width=50)
worksheet.set_row(row=0, height=15)


image_row = 1
image_col = 12
for image in links:
    print(image)
    worksheet.set_row(row=image_row, height=300)
    worksheet.insert_image(image_row,
                          image_col,
                          image,
                          {   'x_scale': .5,
                               'y_scale': .5,
                               'x_offset': 5,
                               'y_offset': 5,
                              'positioning': 1,
                          })
    # positioning = 1 allows move and size with cells (may not always perform as expected)
    image_row += 1

#workbook.filename = f'./data/logged/{gmID}.xlsm'

#workbook.add_vba_project('./vbaProject.bin')


workbook.close()

./images/3a2a78cc-db21-11ee-a158-97f8443fd730/tran_0.jpeg
./images/3a2a78cc-db21-11ee-a158-97f8443fd730/tran_1.jpeg
./images/3a2a78cc-db21-11ee-a158-97f8443fd730/tran_2.jpeg
./images/3a2a78cc-db21-11ee-a158-97f8443fd730/tran_3.jpeg
./images/3a2a78cc-db21-11ee-a158-97f8443fd730/tran_4.jpeg
./images/3a2a78cc-db21-11ee-a158-97f8443fd730/tran_5.jpeg
./images/3a2a78cc-db21-11ee-a158-97f8443fd730/tran_6.jpeg
./images/3a2a78cc-db21-11ee-a158-97f8443fd730/tran_7.jpeg
./images/3a2a78cc-db21-11ee-a158-97f8443fd730/tran_8.jpeg
./images/3a2a78cc-db21-11ee-a158-97f8443fd730/tran_9.jpeg
./images/3a2a78cc-db21-11ee-a158-97f8443fd730/tran_10.jpeg
./images/3a2a78cc-db21-11ee-a158-97f8443fd730/tran_11.jpeg
./images/3a2a78cc-db21-11ee-a158-97f8443fd730/tran_12.jpeg
./images/3a2a78cc-db21-11ee-a158-97f8443fd730/tran_13.jpeg
./images/3a2a78cc-db21-11ee-a158-97f8443fd730/tran_14.jpeg


# Combine with metadata notes
